In [1]:
import tensorflow as tf
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
#ahora importamos lass librerias de ciencias de datos
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()

In [3]:
data=pd.read_excel(r"C:\Users\JOHAN\Documents\DATA-PRUEBA_CALIFICACIONES  2017-2019-FISI.xlsx",sheet_name=None)

In [4]:
data_total=pd.concat([pd.DataFrame(data[key]) for key in data.keys()])

In [5]:
data_total.head()

,cod_semestre,cod_facultad,cod_escuela,cod_plan,cod_asignatura,cod_alumno,val_calific_final
0,20171,20,1,2002,201101,06200023,11
1,20171,20,1,2002,201101,05200188,12
2,20171,20,1,2002,201101,08200095,15
3,20171,20,1,2002,201101,07200077,13
4,20171,20,1,2002,201101,08200198,12


In [6]:
data_total["cod_semestre"].value_counts()

20171    8835
20172    8175
20191    8089
20192    7535
20181    7479
20182    6921
3           1
Name: cod_semestre, dtype: int64

In [7]:
data_total[["cod_semestre","cod_escuela","cod_plan"]]=data_total[["cod_semestre","cod_escuela","cod_plan"]].astype("str")
data_total["cod_semestre"].replace({"3":"20171"},inplace=True)

In [8]:
data_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47035 entries, 0 to 15623
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   cod_semestre       47035 non-null  object
 1   cod_facultad       47035 non-null  int64 
 2   cod_escuela        47035 non-null  object
 3   cod_plan           47035 non-null  object
 4   cod_asignatura     47035 non-null  object
 5   cod_alumno         47035 non-null  object
 6   val_calific_final  47035 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.9+ MB


In [9]:
data_total.drop(["cod_facultad","cod_asignatura"],axis=1,inplace=True)

In [10]:
data_total.drop("cod_alumno",axis=1,inplace=True)

In [13]:
from sklearn.preprocessing import OrdinalEncoder,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

categorical_features=[0,1,2]
categorical_transformer=Pipeline([
    ("imputer",SimpleImputer(strategy="most_frequent")),
    ("encoder",OneHotEncoder(drop="if_binary"))
])


numerical_features=[3]
numerical_transformer=Pipeline([
    ("imputer",SimpleImputer(strategy="mean")),
    ("scaler",MinMaxScaler(feature_range=(0,1)))
])
transformer=ColumnTransformer(
    [("categorical_transformer",categorical_transformer,categorical_features),
     ("numerical_transformer",numerical_transformer,numerical_features)
    ],
    remainder="passthrough"
)

In [14]:
data_transform=transformer.fit_transform(data_total)

In [15]:
data_transform.toarray()


array([[1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.55],
       [1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.6 ],
       [1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.75],
       ...,
       [0.  , 0.  , 0.  , ..., 1.  , 0.  , 0.6 ],
       [0.  , 0.  , 0.  , ..., 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 1.  , 0.  , 0.6 ]])

In [16]:
#graficamos el esquema de codificacion
from sklearn import set_config

set_config(display="diagram")

In [17]:
transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('categorical_transformer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(drop='if_binary'))]),
                                 [0, 1, 2]),
                                ('numerical_transformer',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', MinMaxScaler())]),
                                 [3])])

In [18]:
#involucramos los datos para un modelo de ML
from sklearn.ensemble import RandomForestRegressor

forest=RandomForestRegressor(n_estimators=100,max_depth=10)

In [19]:
model_clf=Pipeline([
    ("preprocess",transformer),
    ("regressor",forest)
])

In [20]:
y=np.random.randint(1,200,size=(47035,))

In [21]:
model_clf.fit(data_total,y)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='if_binary'))]),
                                                  [0, 1, 2]),
                                                 ('numerical_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  [3])])),
                ('regressor', RandomForestRegressor(max_depth=10))])

In [22]:
model_clf.score(data_total,y)

0.010848545038712798

In [23]:
from sklearn.metrics import mean_squared_error
y_pred=model_clf.predict(data_total)
score=mean_squared_error(y,y_pred)

In [24]:
score

3299.7146246364487

In [26]:
#ahora implementamos un modelo con busqueda de mejores hiperparametros
from sklearn.model_selection import GridSearchCV,train_test_split,KFold

x_train,x_test,y_train,y_test=train_test_split(data_total,y,test_size=0.3,random_state=42,shuffle=True)
kf=KFold(n_splits=10,random_state=42,shuffle=True)

grid_model=GridSearchCV(model_clf,
                       param_grid={
                           "regressor__n_estimators":100*np.arange(1,6),
                           "regressor__max_depth":range(1,10)
                       },
                        cv=kf,
                        n_jobs=-1
                       )

In [29]:
grid_model.fit(x_train,y_train)

KeyboardInterrupt: 

In [34]:
for col in data_total.columns[:-1]:
    print(data_total[col].value_counts())

20171    8836
20172    8175
20191    8089
20192    7535
20181    7479
20182    6921
Name: cod_semestre, dtype: int64
1    31519
2    15516
Name: cod_escuela, dtype: int64
2014      15137
2009       8351
2014       7110
2015       6763
2018       3454
2015       3008
2009       2046
2002       1137
PCONV        29
Name: cod_plan, dtype: int64


In [33]:
data_total["cod_plan"]

0        2002  
1        2002  
2        2002  
3        2002  
4        2002  
          ...  
15619      2018
15620      2018
15621      2018
15622      2018
15623      2018
Name: cod_plan, Length: 47035, dtype: object

In [35]:
x_train.shape

(32924, 4)

In [36]:
transformer.fit_transform(x_train).toarray()

array([[0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.9 ],
       [1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.5 ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.7 ],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.75],
       [1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , ..., 0.  , 0.  , 0.45]])

In [44]:
transformer.transformers_["categorical_transformer"]

TypeError: list indices must be integers or slices, not str